# Top of the Atmosphere radiative imbalance
- FSNTOA: Net solar flux at top of atmosphere [W/m2]
- FSNT:   Net solar flux at top of model [W/m2]; positive down
- FLNT:   Net longwave flux at top of model [W/m2]; positive up

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from paths import path_results, path_samoc, file_ex_atm_rcp, file_ex_atm_lpd, file_ex_atm_lpi
from constants import spy
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA

In [ ]:
from paths import CESM_filename

In [ ]:
from constants import A_earth
from xr_regression import xr_quadtrend, xr_lintrend

In [ ]:
ds_rcp = xr.open_dataset(file_ex_atm_rcp, decode_times=False)
ds_lpd = xr.open_dataset(file_ex_atm_lpd, decode_times=False)
ds_lpi = xr.open_dataset(file_ex_atm_lpi, decode_times=False)
dss = [ds_rcp, ds_lpd, ds_lpi]

In [ ]:
f, ax = plt.subplots(3, 3, figsize=(16,16), sharey=True)
for i, ds in enumerate(dss):
    ds.FSNT.plot(ax=ax[i,0])
    ds.FLNT.plot(ax=ax[i,1])
    (ds.FSNT-ds.FLNT).plot(ax=ax[i,2])
plt.tight_layout()

In [ ]:
%%time
# Flux Total Net Top of the model
# positive downward
def TOM_integral(run):
    """ top of the model annual global radiative imbalance """
    len_data = len(IterateOutputCESM(domain='atm', run=run, tavg='yrly', name='FSNT_FLNT'))
    if run in ['ctrl', 'rcp']:  atm_domain = 'atm'
    elif run=='lpd':            atm_domain = 'atm_f09'
    elif run=='lpi':            atm_domain = 'atm_f19'
    AREA = xr_AREA(atm_domain)
    FTNT = np.zeros((len_data))
    times = np.arange(len_data)
    for i, (y,m,s) in enumerate(IterateOutputCESM(domain='atm', run=run, tavg='yrly', name='FSNT_FLNT')):
        print(y)
        ds = xr.open_dataset(s, decode_times=False)
        FTNT[i] = ((ds.FSNT-ds.FLNT)*AREA).sum(dim=('lat', 'lon'))
    da = xr.DataArray(data=FTNT, coords={'time': times}, dims=('time'))
    da.to_netcdf(f'{path_samoc}/TOA/TOM_{run}.nc')
    return da

# FTNT_ctrl = TOM_integral('ctrl')
# FTNT_rcp  = TOM_integral('rcp')
# FTNT_lpd  = TOM_integral('lpd')
# FTNT_lpi  = TOM_integral('lpi')

In [ ]:
FTNT_ctrl = xr.open_dataarray(f'{path_samoc}/TOA/TOM_ctrl.nc', decode_times=False)
FTNT_rcp  = xr.open_dataarray(f'{path_samoc}/TOA/TOM_rcp.nc' , decode_times=False)
FTNT_lpd  = xr.open_dataarray(f'{path_samoc}/TOA/TOM_lpd.nc' , decode_times=False)
FTNT_lpi  = xr.open_dataarray(f'{path_samoc}/TOA/TOM_lpi.nc' , decode_times=False)

In [ ]:
FTNT_dt_ctrl = ((FTNT_ctrl-xr_lintrend(FTNT_ctrl))/A_earth)
FTNT_dt_rcp  = ((FTNT_rcp -xr_quadtrend(FTNT_rcp))/A_earth)
FTNT_dt_lpd  = ((FTNT_lpd -xr_lintrend( FTNT_lpd))/A_earth)
FTNT_dt_lpi  = ((FTNT_lpi -xr_lintrend( FTNT_lpi))/A_earth)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12,5))
for i in range(2):
    ax[i].axhline(0, c='k', lw=.5)
(FTNT_ctrl/A_earth).plot(ax=ax[0])
(FTNT_rcp /A_earth).plot(ax=ax[0])
(FTNT_lpd /A_earth).plot(ax=ax[0])
(FTNT_lpi /A_earth).plot(ax=ax[0])

(FTNT_dt_ctrl).plot(ax=ax[1])
(FTNT_dt_rcp ).plot(ax=ax[1])
(FTNT_dt_lpd ).plot(ax=ax[1])
(FTNT_dt_lpi ).plot(ax=ax[1])



In [ ]:
print(FTNT_dt_ctrl.std().values)
print(FTNT_dt_rcp .std().values)
print(FTNT_dt_lpd .std().values)
print(FTNT_dt_lpi .std().values)

In [ ]:
plt.hist(FTNT_dt_ctrl, alpha=.3, density=True)
plt.hist(FTNT_dt_rcp , alpha=.3, density=True)
plt.hist(FTNT_dt_lpd , alpha=.3, density=True)
plt.hist(FTNT_dt_lpi , alpha=.3, density=True)

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(FTNT_ctrl.time+100, FTNT_ctrl*spy/1e21, lw=2, label='CTRL')
plt.plot(FTNT_rcp .time+200, FTNT_rcp *spy/1e21, lw=2, label='RCP')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('TOM radiative imbalance [ZJ yr$^{-1}$]', fontsize=16)
plt.legend(loc=2, fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/TOA/TOA_global_timeseries')

In [ ]:
# linear detrending
ctrl_lf = np.polyfit(FTNT_ctrl.time, FTNT_ctrl, 1)
rcp_lf  = np.polyfit(FTNT_rcp.time , FTNT_rcp , 1)
ctrl_TOA_lin_detr = FTNT_ctrl-(ctrl_lf[1] + ctrl_lf[0]*FTNT_ctrl.time)
rcp_TOA_lin_detr  = FTNT_rcp -(rcp_lf[1]  + rcp_lf[0] *FTNT_rcp.time )

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl_TOA_lin_detr.time+100, (ctrl_TOA_lin_detr)*spy/1e21, lw=.5, c=f'C{0}')
plt.plot(rcp_TOA_lin_detr .time+200, (rcp_TOA_lin_detr )*spy/1e21, lw=.5, c=f'C{1}')
plt.plot(ctrl_TOA_lin_detr.time+100, (ctrl_TOA_lin_detr.rolling({'time': 5}, center=True).mean())*spy/1e21, lw=2.5, c=f'C{0}', label='CTRL')
plt.plot(rcp_TOA_lin_detr .time+200, (rcp_TOA_lin_detr .rolling({'time': 5}, center=True).mean())*spy/1e21, lw=2.5, c=f'C{1}', label='RCP')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('detr. TOM rad. imbalance [ZJ yr$^{-1}$]', fontsize=16)
plt.legend(loc=2, fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/TOA/TOA_global_timeseries_detr_smoothed')